# Práctica 2

## Ejercicio 1

Consulte el man de fork().
Escriba un proceso p1 que cree tres procesos hijo y espere en un bucle a recibir una señal utilizando la 
función pause(). Incluya un sleep en los procesos hijo para que finalicen unos segundos después de 
ser creados. El proceso padre deberá imprimir un mensaje en pantalla cada vez que finalice un proceso 
hijo, y finalizar cuando lo hayan hecho todos sus hijos.

### Fork(): 

 *creates a new process by duplicating the calling process. The new process is referred to as the child process.  The calling process is referred to as the parent process. The child process and the parent process run in separate memory spaces.  At the time of fork() both memory spaces have the same content.  Memory writes, file mappings (mmap(2)), and unmappings (munmap(2)) performed by one of the processes do not affect the other.*

[ Verl el man](https://man7.org/linux/man-pages/man2/fork.2.html)

**Notas importantes:**
- The child does not inherit timers from its parent.
- On success, the PID of the child process is returned in the parent, and 0 is returned in the child.  On failure, -1 is returned in the parent, no child process is created, and errno is set to indicate the error.

In [ ]:
//Ejemplo de fork()
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
  pid_t pid;
  pid = fork();
  if (pid == 0) {
    printf("Soy el hijo, mi ID es %d y mi padre es %d\n", getpid(), getppid());
  } else if (pid > 0) {
    printf("Soy el padre, mi ID es %d y el ID de mi hijo es %d\n", getpid(), pid);
  } else {
    printf("Error al crear el proceso hijo\n");
  }
  return 0;
}

**SIGCHLD 17** es una señal enviada a un proceso padre cuando uno de sus procesos hijos termina o se detiene. La señal es generada por el sistema operativo y se utiliza para informar al proceso padre de que debe limpiar los recursos asociados con el proceso hijo terminado. Esto puede incluir la liberación de la memoria, el cierre de descriptores de archivos abiertos y otras tareas relacionadas. El proceso padre normalmente recibe esta señal a través de la llamada al sistema wait o waitpid, que bloquea al proceso padre hasta que un proceso hijo termine. El proceso padre luego puede recuperar el estado de salida del proceso hijo terminado y tomar cualquier acción necesaria.

In [23]:
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>

int main(void) {
  pid_t pid;

  pid = fork();
  if (pid == 0) {
    printf("Soy el proceso hijo con PID %d\n", getpid());
    return 0;
  } else {
    int status;
    pid_t child_pid;

    child_pid = wait(&status);
    printf("Soy el proceso padre con PID %d\n", getpid());
    printf("El proceso hijo con PID %d ha terminado\n", child_pid);
    printf("El estado de salida del proceso hijo es %d\n", WEXITSTATUS(status));
  }
  
  return 0;
}

Soy el proceso hijo con PID 131379
Soy el proceso padre con PID 131376
El proceso hijo con PID 131379 ha terminado
El estado de salida del proceso hijo es 0


En este ejemplo, el proceso padre crea un proceso hijo mediante la llamada a fork(). El hijo imprime su PID y termina. El padre, a su vez, espera hasta que el hijo termine con la llamada a wait(), que recibe el estado de salida del hijo y su PID. Finalmente, el padre imprime información sobre el hijo y su estado de salida.

## Ejercicio 2
Consulte el man de wait() y exit(). Repita el Ejercicio 1 utilizando la función wait() en vez de pause() y sin capturar la señal SIGCHLD. Incluya la función exit() para finalizar cada proceso hijo. Cada vez que finalice un proceso hijo, el proceso padre deberá imprimir en pantalla el pid del proceso finalizado y el código de finalización utilizado en el exit() por dicho proceso hijo.



### wait()
 *All of these system calls are used to wait for state changes in a child of the calling process, and obtain information about the child whose state has changed.  A state change is considered to be: the child terminated; the child was stopped by a signal; or the child was resumed by a signal.  In the case of a terminated child, performing a wait allows the system to release the resources associated with the child; if a wait is not performed, then the terminated child remains in a "zombie" state (see NOTES below).*

[ Verl el man](https://man7.org/linux/man-pages/man2/wait.2.html)

**Notas importantes:**
- Nota

### exit()
 *The exit() function causes normal process termination and the least significant byte of status (i.e., status & 0xFF) is returned to the parent (see wait(2)).*

[ Verl el man](https://man7.org/linux/man-pages/man3/exit.3.html)

**Notas importantes:**
- Nota

## Ejercicio 3
Consulte el man de waitpid() y del comando ps. Repita el Ejercicio 2 utilizando la función waitpid()1 en vez de wait(). Compruebe, utilizando el comando ps, cómo aparecen procesos zombie que no aparecían en el Ejercicio 2. Razone acerca de las ventajas/inconvenientes de utilizar waitpid() en vez de wait()
### waitpid()
*La  función  waitpid  suspende  la  ejecución  del  proceso  en  curso  hasta  que un hijo especificado por el argumento pid ha terminado, o hasta que  se  produce  una  señal  cuya acción es finalizar el proceso actual o llamar a la función manejadora de la señal.*

*Si  el  hijo  especificado  por  pid ha terminado cuando se produce la llamada (un proceso "zombie"), la función vuelve inmediatamente. Todos los recursos del sistema reservados por el hijo son liberados.*

[Ver el man](https://man7.org/linux/man-pages/man2/wait.2.html)